<a href="https://colab.research.google.com/github/Albategnius/Python/blob/master/LLM_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# GPT2 from Hugging Face


In [ ]:
from transformers import pipeline

# Load model untuk text generation
generator = pipeline("text-generation", model="gpt2")

# Generate text
result = generator("Hari ini saya ingin belajar", max_length=50, num_return_sequences=1)

print(result[0]['generated_text'])


In [ ]:
from transformers import pipeline

# Contoh summarization
summarizer = pipeline("summarization")
summary = summarizer("Indonesia adalah negara kepulauan...", max_length=30, min_length=10, do_sample=False)
print(summary)


#Simple chatbot

In [ ]:
from transformers import pipeline, set_seed

# Load model GPT-2
chatbot = pipeline('text-generation', model='gpt2')
set_seed(42)  # Untuk hasil yang konsisten

print("=== Chatbot GPT-2 ===")
print("Ketik 'exit' untuk keluar.")

while True:
    user_input = input("Kamu: ")

    if user_input.lower() == 'exit':
        print("Bot: Sampai jumpa!")
        break

    response = chatbot(user_input, max_length=100, num_return_sequences=1, pad_token_id=50256)
    print("Bot:", response[0]['generated_text'].replace(user_input, "").strip())


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

# Load model dan tokenizer Bahasa Indonesia
model_name = "cahya/gpt2-small-indonesian-522M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

# Fungsi chatbot
def chat_with_bot(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=100,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response.replace(prompt, "").strip()

# Chat loop
print("=== Chatbot Bahasa Indonesia ===")
print("Ketik 'exit' untuk keluar.")

while True:
    user_input = input("Kamu: ")
    if user_input.lower() == 'exit':
        print("Bot: Sampai jumpa!")
        break
    response = chat_with_bot(user_input)
    print("Bot:", response)


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

# Load model dan tokenizer Bahasa Indonesia
model_name = "cahya/gpt2-small-indonesian-522M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

# Fungsi untuk merespons berdasarkan history
def chat_with_bot(history, user_input):
    # Gabungkan seluruh percakapan jadi 1 prompt
    full_prompt = history + f"Kamu: {user_input}\nBot:"

    input_ids = tokenizer.encode(full_prompt, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=200,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Ekstrak jawaban bot setelah prompt
    bot_reply = response[len(full_prompt):].strip()
    return bot_reply

# Mulai chatbot
print("=== Chatbot Bahasa Indonesia dengan Memori ===")
print("Ketik 'exit' untuk keluar.")
chat_history = ""

while True:
    user_input = input("Kamu: ")
    if user_input.lower() == "exit":
        print("Bot: Sampai jumpa!")
        break

    bot_response = chat_with_bot(chat_history, user_input)
    print("Bot:", bot_response)

    # Tambahkan ke history
    chat_history += f"Kamu: {user_input}\nBot: {bot_response}\n"
